In [ ]:
import pandas as pd
import numpy as np
np.random.seed(42)
import os
import cv2

import matplotlib.pyplot as plt
import matplotlib as mpl

from utils import load_everything
#from validation import plot_confusion_mat


## Load the Dataset

In [ ]:
# the dataset contains all of the metadata test and train sets are self explanatory 
# no shuffling is required this is all done in the loader. indexes match between the dataset and x_train etc
# to index for the test set do dataset[len(x_train):]
dataset, (x_train, x_test, y_train, y_test) = load_everything(os.path.join('..', 'datasets'))

In [ ]:
dataset.info()

In [ ]:
dataset.head()

In [ ]:
# check if it's within [0, 1] range
print(f'x_train: max({x_train.max()}), min({x_train.min()})\nx_test:  max({x_test.max()}), min({x_test.min()})')

In [ ]:
print(x_train.shape)

In [ ]:
label_stats = pd.concat([
        dataset['fire'].value_counts(),
        dataset['fire'].value_counts(normalize=True)
    ],
    keys=['counts', 'normalized_counts'],
    axis=1,
)
print(label_stats)

In [ ]:
def show_image_samples(dataset):
    labels = np.unique(dataset['fire'])
    fig, axs = plt.subplots(1, len(labels))
    fig.set_size_inches(10,5)
    fig.tight_layout()

    for ax, label in zip(axs, labels):
        img_sample = dataset[dataset['fire'] == label].sample(1)
        img_name = f'{img_sample["img_name"].iloc[0]}.jpg'
        img_path = os.path.join(dataset_base_path, img_name)
        img = plt.imread(img_path)
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(label)

dataset_base_path = os.path.join('../datasets', 'generated')
show_image_samples(dataset)

## Fitting a model to our dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental import preprocessing
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
def get_input_shape(data):
    _, *input_shape = data.shape
    return input_shape

def reshape_data(data, base_size):
    reshaped = [cv2.resize(img, base_size) for img in data]
    return np.array(reshaped)

def standardise(*data):
    data = [d / 255.0 for d in data]
    return data

def add_seq_layers(layers):
    model = models.Sequential([
        *layers
    ])
    return model
            
def plot_model_history(history):
    plt.figure(1, figsize = (15,8)) 
    plt.subplot(221)  
    plt.plot(history.history['acc'])  
    plt.plot(history.history['val_acc'])  
    plt.title('model accuracy')  
    plt.ylabel('accuracy')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'valid']) 

    plt.subplot(222)  
    plt.plot(history.history['loss'])  
    plt.plot(history.history['val_loss'])  
    plt.title('model loss')  
    plt.ylabel('loss')  
    plt.xlabel('epoch')  
    plt.legend(['train', 'valid']) 

    plt.show()

In [ ]:
# VGG16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop

In [ ]:
base_size = (224, 224)
x_train_reshaped, x_test_reshaped = list(map(lambda x: reshape_data(x, base_size), [x_train, x_test]))
print(f'x_train_reshaped: {x_train_reshaped.shape}')

In [ ]:
input_shape = get_input_shape(x_train_reshaped)

vgg16_base_model = VGG16(
    include_top = False, 
    weights = 'imagenet', 
    input_shape = input_shape
)

for layer in vgg16_base_model.layers:
    layer.trainable = False
    
outputs = vgg16_base_model.output
outputs = GlobalAveragePooling2D()(outputs)

outputs = Dense(1024, activation='relu')(outputs)
outputs = Dropout(0.5)(outputs)

outputs = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(outputs)
outputs = Dropout(0.5)(outputs)

outputs = Dense(256, activation='relu')(outputs)
outputs = Dropout(0.5)(outputs)

outputs = Dense(1, activation='sigmoid')(outputs)

vgg16_model = Model(vgg16_base_model.input, outputs)
vgg16_model.compile(
    optimizer = Adam(learning_rate = 0.00001),
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

vgg16_model_history = vgg16_model.fit(
    x_train_reshaped, 
    y_train, 
    validation_data = (x_test_reshaped, y_test), 
    epochs = 50,
    #steps_per_epoch = 100
)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

#Plot the Accuracy and Loss Curves
plot_model_history(vgg16_model_history)

# Plot the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = vgg16_model.predict(x_test_reshaped)
y_pred = np.round(y_pred)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

sns.heatmap(cm, annot=True, fmt='d')



In [ ]:
# Inception
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
base_size = (150, 150)
x_train_reshaped, x_test_reshaped = list(map(lambda x: reshape_data(x, base_size), [x_train, x_test]))
print(f'x_train_reshaped: {x_train_reshaped.shape}')

In [ ]:
input_shape = get_input_shape(x_train_reshaped)
inception_base_model = InceptionV3(
    include_top = False,
    input_shape = input_shape, 
    weights = 'imagenet'
)

for layer in inception_base_model.layers:
    layer.trainable = False
    
outputs = Flatten()(inception_base_model.output)
outputs = Dense(1024, activation='relu')(outputs)
outputs = Dropout(0.5)(outputs)

outputs = Dense(1, activation='sigmoid')(outputs)

inception_model = Model(inception_base_model.input, outputs)

inception_model.compile(
    optimizer = RMSprop(learning_rate = 0.0001), 
    loss = 'binary_crossentropy', 
    metrics = ['acc']
)
inception_model_history = inception_model.fit(
    x_train_reshaped, 
    y_train, 
    validation_data = (x_test_reshaped, y_test), 
    epochs = 100,
    # steps_per_epoch = 100
)

In [ ]:
# plot accuracy and loss
plot_model_history(inception_model_history)

In [ ]:
# plot confusion matrix
y_pred =  np.round(inception_model.predict(x_test_reshaped))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))
sns.heatmap(cm, annot=True, fmt='d')

In [ ]:
# ResNet50
from tensorflow.keras.applications import ResNet50

In [ ]:
base_size = (180, 180)
x_train_reshaped, x_test_reshaped = list(map(lambda x: reshape_data(x, base_size), [x_train, x_test]))
print(f'x_train_reshaped: {x_train_reshaped.shape}')

In [ ]:
input_shape = get_input_shape(x_train_reshaped)
resnet50_base_model = ResNet50(
    include_top = False, 
    weights = 'imagenet', 
    input_shape = input_shape,
    pooling = 'avg',
    classes = 2
)

for model_layer in resnet50_base_model.layers:
    model_layer.trainable = False

In [ ]:
# resnet50_model.compile(
#     optimizer="Adam", 
#     loss="binary_crossentropy", 
#     metrics=["acc"]
# )

# resnet50_model_history = resnet50_model.fit(
#     x_train_reshaped, 
#     y_train, 
#     validation_data = (x_test_reshaped, y_test), 
#     epochs=10
# )